In [4]:
import os
import json
import pandas as pd
import import_ipynb
from utility import *
from extract_test_logs import *

importing Jupyter notebook from utility.ipynb
importing Jupyter notebook from extract_test_logs.ipynb
importing Jupyter notebook from entire_logs.ipynb


In [91]:
def divide_df_per_column(df, column):
    if type(df) == list:
        new_list = []
        for  element_df in df:
            new_list.extend(divide_df_per_column(element_df, column))
        return new_list        
    else:
        values = list(set(df[column]))
        return [df[df[column] == value].copy() for value in values]

def add_answer_pitch(df):
    if type(df) == list:
        return [add_answer_pitch(element_df) for element_df in df]
    else:
        sub_df = df[df["action"] == "StartNote"]
        if len(sub_df) > 0:
            option = sub_df.iloc[0].option
            answer_pitch = option_to_json(option)["pitch"]
        else:
            answer_pitch = 'x'
        df["answer_pitch"] = answer_pitch
        return df

def option_to_column(df, column):
    df = df.copy()
    df[column] = df["option"].apply(lambda x : option_to_json(x)[column])
    return df

In [130]:
test_df = get_test_df()

In [131]:
df_list = [test_df]
df_list = divide_df_per_column(df_list, "teacher")
df_list = divide_df_per_column(df_list, "student")
df_list = divide_df_per_column(df_list, "try")
df_list = divide_df_per_column(df_list, "note_idx")

In [132]:
df_list = add_answer_pitch(df_list)

In [133]:
df_list = [df[df["action"] == "Microphone"] for df in df_list] 
df_list = [option_to_column(df, "pitch") for df in df_list]
df_list = [option_to_column(df, "volume") for df in df_list]

In [141]:
# 예외 처리: answer_pitch가 "x" 인 경우 제거
df_list = [df[df["answer_pitch"] != "x"] for df in df_list]
df_list = [df for df in df_list if 0 < len(df)]

In [163]:
def add_difference_fitch(df):
    df = df.copy()
    df["difference"] = df[["answer_pitch", "pitch", ]].apply(lambda x : MusicManager.to_int(x[0]) - MusicManager.to_int(x[1]), axis=1)
    return df

df_list = [add_difference_fitch(df) for df in df_list]

In [177]:
key_list = ["teacher", "student", "try", "note_idx"]
value_list = ["pitch", "answer_pitch", "difference"]

In [ ]:
df_list2 = [df[key_list+value_list].groupby(key_list).count() for df in df_list]

In [180]:
df_list2[4]

,,,,pitch,answer_pitch,difference
teacher,student,try,note_idx,,,
최민웅_Log,장서영,1,6,117,117,117


In [182]:
df_list3 = [df[key_list+value_list].groupby(key_list).mean() for df in df_list]

In [184]:
df_list3[0]

,,,,difference
teacher,student,try,note_idx,
최민웅_Log,장서영,1,1,3.244681


In [187]:
df = pd.concat(df_list3)

In [191]:
df.reset_index(inplace=True)

In [199]:
df[(df["teacher"] == "최민웅_Log") & (df["student"] == "장서영") & (df["try"] == 1)]

,teacher,student,try,note_idx,difference
0,최민웅_Log,장서영,1,1,3.244681
1,최민웅_Log,장서영,1,2,-9.714286
2,최민웅_Log,장서영,1,4,-4.642336
3,최민웅_Log,장서영,1,5,0.761538
4,최민웅_Log,장서영,1,6,-2.931624
5,최민웅_Log,장서영,1,8,8.015544
6,최민웅_Log,장서영,1,9,-1.476744
7,최민웅_Log,장서영,1,10,-6.952381
8,최민웅_Log,장서영,1,11,2.384615
9,최민웅_Log,장서영,1,13,11.418367


In [203]:
df[(df["teacher"] == "최민웅_Log") & (df["student"] == "장서영") & (df["try"] == 2)]

,teacher,student,try,note_idx,difference
23,최민웅_Log,장서영,2,1,6.110526
24,최민웅_Log,장서영,2,2,2.804878
25,최민웅_Log,장서영,2,3,4.416667
26,최민웅_Log,장서영,2,5,12.562500
27,최민웅_Log,장서영,2,6,7.081081
28,최민웅_Log,장서영,2,7,0.000000
29,최민웅_Log,장서영,2,8,0.000000
30,최민웅_Log,장서영,2,9,2.568182
31,최민웅_Log,장서영,2,10,0.000000
32,최민웅_Log,장서영,2,11,0.000000
